# Configurations

Nightly toolchain is needed for `slas` to work, which is a linear algebra system written by me.

[Plotly](https://lib.rs/plotly) and [itertools](https://lib.rs/itertools) is also needed for plotting data.

In [58]:
:toolchain nightly

:dep slas = { git = "https://github.com/unic0rn9k/slas", features = ["fast-floats"] }
:dep plotly
:dep itertools-num

#![allow(incomplete_features)]
#![feature(generic_const_exprs)]

use slas::prelude::*;
use slas_backend::*;

extern crate plotly;
extern crate rand_distr;
extern crate itertools_num;
extern crate itertools;

use itertools_num::linspace;
use plotly::common::{
    ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode, Title,
};
use plotly::layout::{Axis, BarMode, Layout, Legend, TicksDirection};
use plotly::{Bar, NamedColor, Plot, Rgb, Rgba, Scatter};
use rand_distr::{Distribution, Normal, Uniform};

Toolchain: nightly


# Some example code

For some reasone slas vectors needs type annotations when used in evcxr, even though this is not required normally...

In [52]:
let mut a: StaticCowVec<f32, 3> = moo![f32: 1, 2, 3];
let b: StaticCowVec<f32, 3> = moo![f32: 0..3];

println!("{}", b.static_backend::<Blas>().dot(&a.static_backend::<Blas>()));

8


In [53]:
println!("{a:?}");

[1.0, 2.0, 3.0]


# Defining some useful functions

first we define a function for plotting a vector with index on the x-axis and values on the y-axis

In [54]:
extern crate serde;

fn plot_vector<T: PrimitiveFloat + serde::ser::Serialize + 'static, const LEN: usize>(v: StaticVecRef<T, LEN>){
    let t: Vec<f64> = linspace(0., LEN as f64, LEN).collect();
    let trace = Scatter::new(t, **v).mode(Mode::Markers);
    let mut plot = Plot::new();
    plot.add_trace(trace);
    let layout = Layout::new().height(800);
    plot.set_layout(layout);
    plot.notebook_display();
}

## Generating and plotting a simple wave

In [55]:
use fast_floats::*;

let y: StaticCowVec<f32, 400> = moo![|t|{
    let t = fast(t as f32) / fast(10.);
    *(t.sin_() + (t / fast(3.5)).sin_())
}; 400];

plot_vector(y.moo_ref());

# Euler's identity

In [56]:
im(std::f32::consts::PI).exp_()

(-1 - 0.00000008742278im)

# Cooley-Tukey in python
```python
def FFT(x):
    N = len(x)
    if N <= 1: return x
    even = FFT(x[0::2])
    odd = FFT(x[1::2])
    T = [np.exp(-2j * np.pi * k / N) * odd[k] for k in range(N // 2)]
    return [even[k] + T[k] for k in range(N // 2)] + \
        [even[k] - T[k] for k in range(N // 2)]

```

# Rust translation

In [65]:
use std::f32::consts::PI;

fn fft<const LEN: usize>(x: StaticVecRef<Complex<f32>, LEN>) -> [Complex<f32>; LEN]
    where [(); LEN/2]: Sized
{    
    if LEN < 2 { return **x }
    assert_eq!(LEN % 2, 0);
    
    let even = fft(unsafe{ x.static_slice_unchecked::<2>(0) }.moo_ref());
    let odd = fft(unsafe{ x.static_slice_unchecked::<2>(1) }.moo_ref());
    
    let mut t: [Complex<f32>; LEN/2] = [num!(0); LEN/2];
    for k in 0..LEN/2{
        t[k] = im(-2. * PI) * re(k as f32 / LEN as f32) * odd[k];
    }
    
    todo!();
}